In [1]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *

In [2]:
train = pd.read_csv("./input/train.csv")
test = pd.read_csv("./input/test.csv")

In [3]:
train.shape

(16909, 1301)

In [4]:
train.head()

,Time,X692,X1272,X553,X1299,X4,X598,X1003,X1010,X1216,...,X1164,X656,X1287,X324,X767,X1271,X1288,X58,X1033,X1163
0,2016-07-26 11:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-07-26 12:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-07-26 13:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-07-26 14:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-07-26 15:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


head 찍어보면 다 결측치인데 뭘까..?

In [5]:
train.isnull().sum()

Time         0
X692     13845
X1272    13844
X553     13868
X1299    13903
         ...  
X1271    13844
X1288    13875
X58      15162
X1033    13865
X1163    13868
Length: 1301, dtype: int64

데이터를 보면 모든 미터기의 전력 소비값이 모두 결측치인 데이터가 있는데, 이는 제거하고 분석 진행. 

In [6]:
train.isnull().sum()[(train.isnull().sum() == train.shape[0])].index

Index(['X4', 'X24', 'X64', 'X3', 'X22', 'X45', 'X1', 'X34'], dtype='object')

In [7]:
test.isnull().sum()[(test.isnull().sum() == test.shape[0])].index

Index([], dtype='object')

In [8]:
for i in train.isnull().sum()[(train.isnull().sum() == 16909)].index:
    del train[i]

Time이 계량이 시작한 날짜이고, 그 값부터 값이 존재함 
- 계량의 시작날짜가 같은 것은 같은 아파트라고 할 수 있지 않을까? 
- 상가와 아파트를 분류하기 위해서 평균 전력소비가 큰 그룹과 작은 그룹을 찾을 수 있지 않을까? 

In [9]:
train.tail()

,Time,X692,X1272,X553,X1299,X598,X1003,X1010,X1216,X1047,...,X1164,X656,X1287,X324,X767,X1271,X1288,X58,X1033,X1163
16904,2018-06-30 19:00,0.271,0.754,0.327,0.468,0.314,0.769,0.790,0.558,0.149,...,0.523,0.455,0.295,0.056,0.295,2.089,0.619,1.063,0.345,0.565
16905,2018-06-30 20:00,0.341,0.827,0.478,0.356,0.434,0.680,0.819,0.470,0.160,...,0.350,0.799,0.287,0.057,0.548,2.168,0.723,4.760,0.388,0.620
16906,2018-06-30 21:00,0.535,0.516,0.558,0.496,0.479,0.590,0.910,0.424,0.468,...,0.446,0.754,0.433,0.060,0.677,2.028,0.766,5.386,0.440,0.573
16907,2018-06-30 22:00,0.379,0.494,0.605,0.320,0.555,0.604,1.044,0.412,0.480,...,0.567,0.504,0.384,0.210,1.019,1.055,0.735,7.150,0.393,0.629
16908,2018-06-30 23:00,0.388,0.482,0.651,0.284,0.452,0.629,1.116,0.400,0.274,...,0.495,0.410,0.343,0.420,0.746,0.738,0.689,8.346,0.323,0.422


In [10]:
test.tail()

,Time,X26,X303,X241,X435,X402,X352,X305,X350,X326,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
8755,2018.6.30 19:00,1.255,0.175,0.430,0.429,0.169,0.739,0.573,0.341,0.127,...,0.312,0.381,0.479,0.858,11.546,0.428,0.134,3.519,0.027,0.286
8756,2018.6.30 20:00,1.596,1.692,0.332,0.366,0.159,0.255,0.501,0.281,0.126,...,0.327,0.402,0.725,0.816,11.591,0.413,0.206,3.331,0.027,0.303
8757,2018.6.30 21:00,1.379,0.883,0.435,0.497,0.166,0.233,0.634,0.401,0.439,...,0.253,0.401,0.426,0.659,10.766,0.466,0.324,3.416,0.026,0.253
8758,2018.6.30 22:00,1.000,0.539,0.268,0.459,0.463,0.454,0.676,0.543,0.737,...,0.332,0.401,0.353,0.462,0.312,0.473,0.402,4.263,0.027,0.253
8759,2018.6.30 23:00,0.976,0.481,0.220,0.418,0.646,0.419,0.680,0.499,0.304,...,0.434,0.435,0.349,0.792,0.309,0.519,0.334,0.784,0.027,0.310


In [11]:
sub = pd.read_csv("./input/submission_1002.csv")
sub

,meter_id,X2018_7_1_1h,X2018_7_1_2h,X2018_7_1_3h,X2018_7_1_4h,X2018_7_1_5h,X2018_7_1_6h,X2018_7_1_7h,X2018_7_1_8h,X2018_7_1_9h,...,X2018_7_6_d,X2018_7_7_d,X2018_7_8_d,X2018_7_9_d,X2018_7_10_d,X2018_7_m,X2018_8_m,X2018_9_m,X2018_10_m,X2018_11_m
0,X5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,X7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,X10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,X16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,X18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,X468,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196,X470,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197,X472,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
198,X473,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


train과 test의 간격은 동일하고, submission은 test 이후의 기간만을 포함하고 있음. 
- 그렇다면 도대체 Train은 왜 준 것일까..? 
- 어떻게 활용해야 할까? 

### 데이터 탐색

In [18]:
train_T = train.T
test_T = test.T

train_T.columns = train_T.iloc[0]
train_T = train_T.iloc[1:]

test_T.columns = test_T.iloc[0]
test_T = test_T.iloc[1:]

print(train_T.shape, test_T.shape)

(1292, 16909) (200, 8760)


In [19]:
train_T.head()

Time,2016-07-26 11:00,2016-07-26 12:00,2016-07-26 13:00,2016-07-26 14:00,2016-07-26 15:00,2016-07-26 16:00,2016-07-26 17:00,2016-07-26 18:00,2016-07-26 19:00,2016-07-26 20:00,...,2018-06-30 14:00,2018-06-30 15:00,2018-06-30 16:00,2018-06-30 17:00,2018-06-30 18:00,2018-06-30 19:00,2018-06-30 20:00,2018-06-30 21:00,2018-06-30 22:00,2018-06-30 23:00
X692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.275,0.328,0.498,0.396,0.331,0.271,0.341,0.535,0.379,0.388
X1272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.688,0.484,0.543,0.631,0.666,0.754,0.827,0.516,0.494,0.482
X553,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.45,0.454,0.468,0.397,0.415,0.327,0.478,0.558,0.605,0.651
X1299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.159,0.18,0.171,0.17,0.312,0.468,0.356,0.496,0.32,0.284
X598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.351,0.445,0.344,0.475,0.288,0.314,0.434,0.479,0.555,0.452


In [20]:
test_T.head()

Time,2017.7.1 0:00,2017.7.1 1:00,2017.7.1 2:00,2017.7.1 3:00,2017.7.1 4:00,2017.7.1 5:00,2017.7.1 6:00,2017.7.1 7:00,2017.7.1 8:00,2017.7.1 9:00,...,2018.6.30 14:00,2018.6.30 15:00,2018.6.30 16:00,2018.6.30 17:00,2018.6.30 18:00,2018.6.30 19:00,2018.6.30 20:00,2018.6.30 21:00,2018.6.30 22:00,2018.6.30 23:00
X26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.99,1.179,1.703,1.504,1.271,1.255,1.596,1.379,1,0.976
X303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.142,0.211,0.168,0.14,0.495,0.175,1.692,0.883,0.539,0.481
X241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.319,0.245,0.211,0.215,0.372,0.43,0.332,0.435,0.268,0.22
X435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.261,0.208,0.207,0.199,0.331,0.429,0.366,0.497,0.459,0.418
X402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.213,0.188,0.179,0.213,0.232,0.169,0.159,0.166,0.463,0.646


In [21]:
print(train_T.shape, test_T.shape)

(1292, 16909) (200, 8760)


In [22]:
train_T.isnull().sum()

Time
2016-07-26 11:00    1281
2016-07-26 12:00    1281
2016-07-26 13:00    1281
2016-07-26 14:00    1281
2016-07-26 15:00    1281
                    ... 
2018-06-30 19:00      92
2018-06-30 20:00      92
2018-06-30 21:00      92
2018-06-30 22:00      92
2018-06-30 23:00      92
Length: 16909, dtype: int64

In [23]:
test_T.isnull().sum()

Time
2017.7.1 0:00      176
2017.7.1 1:00      176
2017.7.1 2:00      176
2017.7.1 3:00      176
2017.7.1 4:00      176
                  ... 
2018.6.30 19:00      0
2018.6.30 20:00      0
2018.6.30 21:00      0
2018.6.30 22:00      0
2018.6.30 23:00      0
Length: 8760, dtype: int64

* 주의: 제공되는 데이터에는 결측치나 이상치(NA, 0인 값)가 포함되어 있습니다. 대회참가자들은 이러한 결측치를 고려하여, 결측치 처리 등 예측 기법을 적용하여야 합니다.

- 참고로, NA가 발생한 경우, 직전 시간의 전력사용량 값이 상당히 큰 경향이 있습니다. 이는 미터링 데이터 수집 시스템의 특징으로 보입니다. 그러나 반드시 그런 것은 아닙니다.

In [25]:
from tqdm import tqdm_notebook 

meter_id_train = [c for c in train.columns if c not in ['Time']]
meter_id_test = [c for c in test.columns if c not in ['Time']]

train_index = []
for i in tqdm_notebook(meter_id_train):
    #index 추출
    train_index.append(train[train[i].notnull() == True]['Time'].iloc[0])
    
test_index = []
for i in tqdm_notebook(meter_id_test):
    #index 추출
    test_index.append(test[test[i].notnull() == True]['Time'].iloc[0])
    
train_T['first_visit'] = train_index
test_T['first_visit'] = test_index

In [26]:
test_T

Time,2017.7.1 0:00,2017.7.1 1:00,2017.7.1 2:00,2017.7.1 3:00,2017.7.1 4:00,2017.7.1 5:00,2017.7.1 6:00,2017.7.1 7:00,2017.7.1 8:00,2017.7.1 9:00,...,2018.6.30 15:00,2018.6.30 16:00,2018.6.30 17:00,2018.6.30 18:00,2018.6.30 19:00,2018.6.30 20:00,2018.6.30 21:00,2018.6.30 22:00,2018.6.30 23:00,first_visit
X26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.179,1.703,1.504,1.271,1.255,1.596,1.379,1,0.976,2018.4.18 19:00
X303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.211,0.168,0.14,0.495,0.175,1.692,0.883,0.539,0.481,2017.8.25 0:00
X241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.245,0.211,0.215,0.372,0.43,0.332,0.435,0.268,0.22,2017.8.25 0:00
X435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.208,0.207,0.199,0.331,0.429,0.366,0.497,0.459,0.418,2017.8.29 10:00
X402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.188,0.179,0.213,0.232,0.169,0.159,0.166,0.463,0.646,2017.8.29 10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.378,0.459,0.388,0.406,0.428,0.413,0.466,0.473,0.519,2017.8.29 10:00
X347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.197,0.232,0.252,0.244,0.134,0.206,0.324,0.402,0.334,2017.8.29 10:00
X75,0.275,0.222,0.237,0.229,0.202,0.251,0.239,0.207,0.176,0.733,...,4.813,4.38,4.337,3.188,3.519,3.331,3.416,4.263,0.784,2017.7.1 0:00
X107,0.021,0.021,0.021,0.02,0.021,0.021,0.021,0.02,0.021,0.021,...,0.027,0.027,0.027,0.027,0.027,0.027,0.026,0.027,0.027,2017.7.1 0:00


In [80]:
sub[sub['meter_id'] == 'X26']

,meter_id,X2018_7_1_1h,X2018_7_1_2h,X2018_7_1_3h,X2018_7_1_4h,X2018_7_1_5h,X2018_7_1_6h,X2018_7_1_7h,X2018_7_1_8h,X2018_7_1_9h,...,X2018_7_6_d,X2018_7_7_d,X2018_7_8_d,X2018_7_9_d,X2018_7_10_d,X2018_7_m,X2018_8_m,X2018_9_m,X2018_10_m,X2018_11_m
6,X26,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


1. 상가 / 아파트 분류 
2. tabular form 만들어서 boosting 모델 적용 
   -  날씨데이터 이용해서 예측 
3. ARIMA Parameter 손보기 
    - RNN 
    - LSTM 
4. Facebook Prophet 